<a href="https://colab.research.google.com/github/KYK0328/ecommerce/blob/master/productlist_popularity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 초기 설정

In [ ]:
#구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=7f903d77d741717b692fc3a2aae58dc73408477670ec8b795e4388d2a5c268c1
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [ ]:
import pyspark
from pyspark import SparkConf
from pyspark import SparkContext

from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
import os
os.chdir('/content/drive/MyDrive/cosmetic')

In [ ]:
new_df = spark.read.option('header', True).option('inferSchema', True).csv('/content/drive/MyDrive/cosmetic/new.csv')
total = new_df.createOrReplaceTempView('total')

## 1. 데이터 확인
- 월별 코호트 구분
- 10월, 1월 코호트의 (최다 구매 제품, 브랜드) 특성 파악
- event_type= purchase '구매'에 초점

In [ ]:
result = spark.sql("""
SELECT *
FROM total
Limit 10
""")
result.show()

+-------------------+----------+----------+-------------------+-------------+------+-----+-------+------------+
|         event_time|event_type|product_id|        category_id|category_code| brand|price|user_id|user_session|
+-------------------+----------+----------+-------------------+-------------+------+-----+-------+------------+
|2020-01-14 16:15:21|      view|   5865526|1487580008447738866|         null|   cnd| 10.0| 465496|           1|
|2020-01-14 16:25:35|      view|   5769989|1487580008447738866|         null|   cnd| 10.0| 465496|           2|
|2020-01-14 16:27:31|      view|   5865524|1487580008447738866|         null|   cnd| 10.0| 465496|           2|
|2019-12-22 12:50:22|      view|   5746011|1487580009051717646|         null|runail|34.92|2963072|          38|
|2019-12-22 12:50:45|      view|   5707747|1487580009051717646|         null|  null|73.02|2963072|          38|
|2019-12-22 12:50:58|      view|   5746011|1487580009051717646|         null|runail|34.92|2963072|      

### 코호트 preprocessing

In [ ]:
result = spark.sql("""
WITH stats AS (
  SELECT user_id
       , DATE_FORMAT(MIN(event_time), 'yyyy-MM-dd') AS first_order_date
       , DATE_FORMAT(MAX(event_time), 'yyyy-MM-dd') AS last_order_date
  FROM total
  WHERE event_type = 'purchase'
  GROUP BY user_id
), processed AS (
  SELECT t.user_id
       , DATE_FORMAT(s.first_order_date, 'yyyy-MM-01') AS first_order_month
       , DATE_FORMAT(t.event_time, 'yyyy-MM-dd') AS order_date
       , s.first_order_date
  FROM total t
    INNER JOIN stats s ON t.user_id = s.user_id AND t.event_type = 'purchase'
)
SELECT *
FROM processed;

""")
result.show()

+--------+-----------------+----------+----------------+
| user_id|first_order_month|order_date|first_order_date|
+--------+-----------------+----------+----------------+
|43713532|       2019-10-01|2019-10-13|      2019-10-13|
|43713532|       2019-10-01|2019-10-13|      2019-10-13|
|43713532|       2019-10-01|2019-10-13|      2019-10-13|
|43713532|       2019-10-01|2019-10-13|      2019-10-13|
|43713532|       2019-10-01|2019-10-13|      2019-10-13|
|43713532|       2019-10-01|2019-10-13|      2019-10-13|
|43713532|       2019-10-01|2019-10-13|      2019-10-13|
|43713532|       2019-10-01|2019-10-13|      2019-10-13|
|43713532|       2019-10-01|2019-10-13|      2019-10-13|
|43713532|       2019-10-01|2019-10-13|      2019-10-13|
|43713532|       2019-10-01|2019-10-13|      2019-10-13|
|43713532|       2019-10-01|2019-10-13|      2019-10-13|
|43713532|       2019-10-01|2019-10-13|      2019-10-13|
|43713532|       2019-10-01|2019-10-13|      2019-10-13|
|43713532|       2019-10-01|201

### 10월 코호트

In [ ]:
# 10월 코호트의 최다 구매
result = spark.sql("""
WITH stats AS (
  SELECT user_id
        , product_id
        , price
        , brand
       , DATE_FORMAT(MIN(event_time), 'yyyy-MM-dd') AS first_order_date
       , DATE_FORMAT(MAX(event_time), 'yyyy-MM-dd') AS last_order_date
  FROM total
  WHERE event_type = 'purchase'
  GROUP BY user_id, product_id, price, brand
), processed AS (
  SELECT t.user_id
        , t.product_id
        , t.price
        , t.brand
       , DATE_FORMAT(s.first_order_date, 'yyyy-MM-01') AS first_order_month
       , DATE_FORMAT(t.event_time, 'yyyy-MM-dd') AS order_date
       , s.first_order_date
  FROM total t
    INNER JOIN stats s ON t.user_id = s.user_id AND t.event_type = 'purchase'
)
SELECT product_id
      , price
      , brand
      , COUNT(DISTINCT user_id) AS user_count
FROM processed
WHERE first_order_month = '2019-10-01'
GROUP BY product_id, price, brand
ORDER BY user_count DESC;
""")
result.show()

+----------+-----+---------+----------+
|product_id|price|    brand|user_count|
+----------+-----+---------+----------+
|   5809910| 5.24|  grattol|      1298|
|   5854897| 0.32|    irisk|      1095|
|   5700037|  0.4|   runail|       954|
|   5751422|10.95|      uno|       953|
|   5802432| 0.32|     null|       904|
|   5792800|10.32|     null|       796|
|   5751383|10.32|      uno|       788|
|   5815662| 0.92|     null|       773|
|   5809912| 5.24|  grattol|       770|
|   5686925| 0.35|     null|       751|
|      5304| 0.32|   runail|       729|
|   5849033|10.32|      uno|       678|
|   5687151|  1.9|     null|       563|
|      5013| 1.27|   runail|       523|
|   5688124| 0.32|     null|       507|
|   5528035| 9.52|     null|       483|
|   5833330| 0.95|bpw.style|       474|
|   5700046|  0.4|   runail|       471|
|   5809911| 5.24|  grattol|       465|
|   5649219| 7.14|     null|       456|
+----------+-----+---------+----------+
only showing top 20 rows



In [ ]:
# 10월 코호트의 10월 구매 item 확인
result = spark.sql("""
WITH stats AS (
  SELECT user_id
        , product_id
        , price
        , brand
       , DATE_FORMAT(MIN(event_time), 'yyyy-MM-dd') AS first_order_date
       , DATE_FORMAT(MAX(event_time), 'yyyy-MM-dd') AS last_order_date
  FROM total
  WHERE event_type = 'purchase'
  GROUP BY user_id, product_id, price, brand
), processed AS (
  SELECT t.user_id
        , t.product_id
        , t.price
        , t.brand
       , DATE_FORMAT(s.first_order_date, 'yyyy-MM-01') AS first_order_month
       , DATE_FORMAT(t.event_time, 'yyyy-MM-dd') AS order_date
       , s.first_order_date
  FROM total t
    INNER JOIN stats s ON t.user_id = s.user_id AND t.event_type = 'purchase'
)
SELECT product_id
      , price
      , brand
      , COUNT(DISTINCT product_id) AS product_c
FROM processed
WHERE first_order_month = '2019-10-01'
GROUP BY product_id, price, brand
ORDER BY product_c DESC;
""")
result.show()

+----------+-----+--------+---------+
|product_id|price|   brand|product_c|
+----------+-----+--------+---------+
|   5810157| 1.43|   irisk|        1|
|   5854574|21.43|jessnail|        1|
|   5676826| 4.29|   estel|        1|
|   5869121|15.56|    oniq|        1|
|   5823937|94.44| polarus|        1|
|   5859398| 6.33| lianail|        1|
|   5798924| 3.97|  zinger|        1|
|   5811649| 2.48|   irisk|        1|
|   5836515| 0.08|nagaraku|        1|
|   5785197| 9.51| markell|        1|
|   5881615|12.56|  lovely|        1|
|   5787899| 4.29|  uskusi|        1|
|   5723511| 2.62|  runail|        1|
|   5677986| 7.14|   estel|        1|
|   5867123| 2.83|   domix|        1|
|   5804045| 3.97| bluesky|        1|
|   5774328| 3.81|  runail|        1|
|   5564074| 2.38|ingarden|        1|
|   5866963|66.67|   sunuv|        1|
|   5830525| 3.95|  masura|        1|
+----------+-----+--------+---------+
only showing top 20 rows



In [ ]:
result = spark.sql("""
WITH stats AS (
 SELECT user_id
        , product_id
        , price
        , brand
       , DATE_FORMAT(MIN(event_time), 'yyyy-MM-dd') AS first_order_date
       , DATE_FORMAT(MAX(event_time), 'yyyy-MM-dd') AS last_order_date
  FROM total
  WHERE event_type = 'purchase'
  GROUP BY user_id, product_id, price, brand
),
processed AS (
  SELECT t.user_id
        , t.product_id
        , t.price
        , t.brand
        , DATE_FORMAT(s.first_order_date, 'yyyy-MM-01') AS first_order_month
        , DATE_FORMAT(t.event_time, 'yyyy-MM-dd') AS order_date
        , s.first_order_date
  FROM total t
    INNER JOIN stats s ON t.user_id = s.user_id AND t.event_type = 'purchase'
)
SELECT product_id
      , price
      , brand
      , COUNT(DISTINCT user_id) AS user_count
FROM processed
WHERE first_order_month = '2019-10-01'
GROUP BY product_id, price, brand
ORDER BY user_count DESC;

""")
result.show()

+----------+-----+---------+----------+
|product_id|price|    brand|user_count|
+----------+-----+---------+----------+
|   5809910| 5.24|  grattol|      1298|
|   5854897| 0.32|    irisk|      1095|
|   5700037|  0.4|   runail|       954|
|   5751422|10.95|      uno|       953|
|   5802432| 0.32|     null|       904|
|   5792800|10.32|     null|       796|
|   5751383|10.32|      uno|       788|
|   5815662| 0.92|     null|       773|
|   5809912| 5.24|  grattol|       770|
|   5686925| 0.35|     null|       751|
|      5304| 0.32|   runail|       729|
|   5849033|10.32|      uno|       678|
|   5687151|  1.9|     null|       563|
|      5013| 1.27|   runail|       523|
|   5688124| 0.32|     null|       507|
|   5528035| 9.52|     null|       483|
|   5833330| 0.95|bpw.style|       474|
|   5700046|  0.4|   runail|       471|
|   5809911| 5.24|  grattol|       465|
|   5649219| 7.14|     null|       456|
+----------+-----+---------+----------+
only showing top 20 rows



In [ ]:
# 10월 코호트 구매리스트 뽑기
result = spark.sql("""
SELECT user_id
        , product_id
        , price
        , brand
       , DATE_FORMAT(MIN(event_time), 'yyyy-MM-dd') AS first_order_date
FROM total
WHERE event_type = 'purchase' AND event_time <= '2019-10-31'
GROUP BY user_id, product_id, price, brand;
""")
result.show()

+---------+----------+-----+----------+----------------+
|  user_id|product_id|price|     brand|first_order_date|
+---------+----------+-----+----------+----------------+
|255774180|   5802162|23.81|   italwax|      2019-10-23|
|271555056|   5849033|10.32|       uno|      2019-10-14|
|297933754|   5855507|79.21|       max|      2019-10-11|
|301930560|   5871037| 3.89|  siberina|      2019-10-20|
|339640418|   5848895| 0.79| bpw.style|      2019-10-08|
|372246203|   5816171| 5.24|   grattol|      2019-10-07|
|374798083|   5743974| 1.98|   italwax|      2019-10-02|
|377153707|   5820270|26.83|       uno|      2019-10-28|
|386627935|   5818359| 2.14|igrobeauty|      2019-10-24|
|392293132|      6691| 0.87|    runail|      2019-10-23|
|407591551|   5809911| 5.24|   grattol|      2019-10-30|
|408963779|   5739056| 6.27|    kapous|      2019-10-16|
|414072562|     58865| 1.27|    runail|      2019-10-29|
|446496752|   5804261|15.71|       cnd|      2019-10-26|
|459529883|   5783524| 6.03|   

In [ ]:
# 10월 코호트 구매 리스트에서 브랜드별 총판매수
result = spark.sql("""
WITH first_orders AS (
  SELECT user_id
        , product_id
        , price
        , brand
        , MIN(event_time) AS first_order_time
  FROM total
  WHERE event_type = 'purchase' AND event_time <= '2019-10-31'
  GROUP BY user_id, product_id, price, brand
)
SELECT brand
      , COUNT(*) AS brand_sales_count
FROM first_orders
GROUP BY brand
ORDER BY brand_sales_count DESC;

""")
result.show()

+---------+-----------------+
|    brand|brand_sales_count|
+---------+-----------------+
|     null|            95014|
|   runail|            20733|
|    irisk|            14664|
|   masura|            10496|
|bpw.style|             9131|
|  grattol|             5998|
| ingarden|             4888|
|  italwax|             3307|
|     pole|             3271|
|    estel|             3059|
|      uno|             2990|
|   kapous|             2917|
|    domix|             2633|
|  bluesky|             2418|
|freedecor|             2392|
|     milv|             2292|
| haruyama|             2284|
|      cnd|             2193|
|   zinger|             1870|
|  concept|             1764|
+---------+-----------------+
only showing top 20 rows



In [ ]:
result = spark.sql("""
WITH stats AS (
  SELECT user_id
       , product_id
       , price
       , brand
       , DATE_FORMAT(MIN(event_time), 'yyyy-MM-dd') AS first_order_date
       , DATE_FORMAT(MAX(event_time), 'yyyy-MM-dd') AS last_order_date
  FROM total
  WHERE event_type = 'purchase'
  GROUP BY user_id, product_id, price, brand
), processed AS (
  SELECT t.user_id
        , t.product_id
        , t.price
        , t.brand
       , DATE_FORMAT(s.first_order_date, 'yyyy-MM-01') AS first_order_month
       , DATE_FORMAT(t.event_time, 'yyyy-MM-dd') AS order_date
       , first_order_date
  FROM total t
    INNER JOIN stats s ON t.user_id = s.user_id AND t.event_type = 'purchase'
)
SELECT first_order_month
     , COUNT(DISTINCT user_id) AS day0


FROM processed
WHERE first_order_month = '2019-10-01'
GROUP BY first_order_month
""")
result.show()

+-----------------+-----+
|first_order_month| day0|
+-----------------+-----+
|       2019-10-01|25762|
+-----------------+-----+



In [ ]:
result = spark.sql("""
WITH stats AS (
  SELECT user_id
       , product_id
       , price
       , brand
       , DATE_FORMAT(MIN(event_time), 'yyyy-MM-dd') AS first_order_date
       , DATE_FORMAT(MAX(event_time), 'yyyy-MM-dd') AS last_order_date
  FROM total
  WHERE event_type = 'purchase'
  GROUP BY user_id, product_id, price, brand
), processed AS (
  SELECT t.user_id
        , t.product_id
        , t.price
        , t.brand
       , DATE_FORMAT(s.first_order_date, 'yyyy-MM-01') AS first_order_month
       , DATE_FORMAT(t.event_time, 'yyyy-MM-dd') AS order_date
       , first_order_date
  FROM total t
    INNER JOIN stats s ON t.user_id = s.user_id AND t.event_type = 'purchase'
  WHERE order_date <= '2019-10-31'
)
SELECT order_date
     ,product_id
     , price
     , brand
     , COUNT(DISTINCT product_id) AS pd
FROM processed
WHERE order_date <= '2019-10-31'
GROUP BY order_date, product_id, price, brand
""")
result.show()

+----------+----------+-----+---------+---+
|order_date|product_id|price|    brand| pd|
+----------+----------+-----+---------+---+
|2019-10-12|   5830533| 1.73|   masura|  1|
|2019-10-04|   5700035|  0.4|   runail|  1|
|2019-10-22|   5839650| 2.37|   masura|  1|
|2019-10-25|   5809270| 1.43|bpw.style|  1|
|2019-10-22|     36549| 5.08| ingarden|  1|
|2019-10-10|   5779806| 4.11|     pole|  1|
|2019-10-15|     19162|10.29|      cnd|  1|
|2019-10-12|   5692024| 5.32|  concept|  1|
|2019-10-16|      5581| 0.56|   entity|  1|
|2019-10-14|   5659645| 0.95| airnails|  1|
|2019-10-16|   5782125| 4.11|     pole|  1|
|2019-10-29|   5816170| 5.24|  grattol|  1|
|2019-10-29|   5678037| 7.14|    estel|  1|
|2019-10-28|   5859407| 2.37|   masura|  1|
|2019-10-09|   5786767| 5.54|  lianail|  1|
|2019-10-07|   5829347| 0.63| farmstay|  1|
|2019-10-17|   5677245|  7.7|    estel|  1|
|2019-10-27|   5724647| 2.48|    irisk|  1|
|2019-10-07|      4958| 1.19|   runail|  1|
|2019-10-12|   5864643|  1.9|bpw

In [ ]:
result = spark.sql("""
WITH stats AS (
  SELECT user_id
       , product_id
       , price
       , brand
       , DATE_FORMAT(MIN(event_time), 'yyyy-MM-dd') AS first_order_date
       , DATE_FORMAT(MAX(event_time), 'yyyy-MM-dd') AS last_order_date
  FROM total
  WHERE event_type = 'purchase'
  GROUP BY user_id, product_id, price, brand
), processed AS (
  SELECT t.user_id
        , t.product_id
        , t.price
        , t.brand
        , DATE_FORMAT(s.first_order_date, 'yyyy-MM-01') AS first_order_month
        , DATE_FORMAT(t.event_time, 'yyyy-MM-dd') AS order_date
        , s.first_order_date
  FROM total t
    INNER JOIN stats s ON t.user_id = s.user_id AND t.event_type = 'purchase'
  WHERE DATE_FORMAT(t.event_time, 'yyyy-MM-dd') <= '2019-10-31'
)
SELECT product_id
     , price
     , brand
     , COUNT(DISTINCT user_id) AS pd
FROM processed
GROUP BY product_id, price, brand
ORDER BY pd DESC
""")
result.show()

+----------+-----+---------+---+
|product_id|price|    brand| pd|
+----------+-----+---------+---+
|   5854897| 0.32|    irisk|672|
|   5700037|  0.4|   runail|671|
|   5751422|10.95|      uno|619|
|   5802432| 0.32|     null|612|
|   5686925| 0.35|     null|565|
|   5792800|10.32|     null|556|
|   5751383|10.32|      uno|540|
|      5304| 0.32|   runail|509|
|   5815662| 0.92|     null|475|
|   5688124| 0.32|     null|448|
|   5849033|10.32|      uno|421|
|   5528035| 9.52|     null|379|
|      5013| 1.27|   runail|352|
|   5729864| 0.41|     null|345|
|   5833334| 0.79|bpw.style|335|
|   5759492| 2.37|  italwax|326|
|   5700046|  0.4|   runail|323|
|   5809912| 5.24|  grattol|322|
|      5581| 0.56|   entity|318|
|   5809910| 5.24|  grattol|302|
+----------+-----+---------+---+
only showing top 20 rows



In [ ]:
# 10월 코호트의 상품 구매리스트에서 최다 구매 상품 추출
# 근데 문제는 1400
result = spark.sql("""
WITH stats AS (
  SELECT user_id
       , product_id
       , price
       , brand
       , DATE_FORMAT(MIN(event_time), 'yyyy-MM-dd') AS first_order_date
       , DATE_FORMAT(MAX(event_time), 'yyyy-MM-dd') AS last_order_date
  FROM total
  WHERE event_type = 'purchase'
  GROUP BY user_id, product_id, price, brand
), processed AS (
  SELECT t.user_id
        , t.product_id
        , t.price
        , t.brand
        , DATE_FORMAT(s.first_order_date, 'yyyy-MM-01') AS first_order_month
        , DATE_FORMAT(t.event_time, 'yyyy-MM-dd') AS order_date
        , s.first_order_date
  FROM total t
    INNER JOIN stats s ON t.user_id = s.user_id AND t.event_type = 'purchase'
  WHERE DATE_FORMAT(s.first_order_date, 'yyyy-MM-01') = '2019-10-01'
)
SELECT product_id
     , price
     , brand
     , COUNT(DISTINCT user_id) AS pd
FROM processed
GROUP BY product_id, price, brand
ORDER BY pd DESC
""")
result.show()

+----------+-----+---------+----+
|product_id|price|    brand|  pd|
+----------+-----+---------+----+
|   5809910| 5.24|  grattol|1298|
|   5854897| 0.32|    irisk|1095|
|   5700037|  0.4|   runail| 954|
|   5751422|10.95|      uno| 953|
|   5802432| 0.32|     null| 904|
|   5792800|10.32|     null| 796|
|   5751383|10.32|      uno| 788|
|   5815662| 0.92|     null| 773|
|   5809912| 5.24|  grattol| 770|
|   5686925| 0.35|     null| 751|
|      5304| 0.32|   runail| 729|
|   5849033|10.32|      uno| 678|
|   5687151|  1.9|     null| 563|
|      5013| 1.27|   runail| 523|
|   5688124| 0.32|     null| 507|
|   5528035| 9.52|     null| 483|
|   5833330| 0.95|bpw.style| 474|
|   5700046|  0.4|   runail| 471|
|   5809911| 5.24|  grattol| 465|
|   5649219| 7.14|     null| 456|
+----------+-----+---------+----+
only showing top 20 rows



### 1월 코호트

In [ ]:
# 1월 코호트의 상품 구매리스트에서 최다 구매 상품 추출
result = spark.sql("""
WITH stats AS (
  SELECT user_id
       , product_id
       , price
       , brand
       , DATE_FORMAT(MIN(event_time), 'yyyy-MM-dd') AS first_order_date
       , DATE_FORMAT(MAX(event_time), 'yyyy-MM-dd') AS last_order_date
  FROM total
  WHERE event_type = 'purchase'
  GROUP BY user_id, product_id, price, brand
), processed AS (
  SELECT t.user_id
        , t.product_id
        , t.price
        , t.brand
        , DATE_FORMAT(s.first_order_date, 'yyyy-MM-01') AS first_order_month
        , DATE_FORMAT(t.event_time, 'yyyy-MM-dd') AS order_date
        , s.first_order_date
  FROM total t
    INNER JOIN stats s ON t.user_id = s.user_id AND t.event_type = 'purchase'
  WHERE DATE_FORMAT(s.first_order_date, 'yyyy-MM-01') = '2020-01-01'
)
SELECT  product_id
     , price
     , brand
     , COUNT(DISTINCT user_id) AS pd
FROM processed
GROUP BY product_id, price, brand
ORDER BY pd DESC
""")
result.show()

+----------+-----+-------+----+
|product_id|price|  brand|  pd|
+----------+-----+-------+----+
|   5809910| 5.24|grattol|2716|
|   5854897| 0.32|  irisk|1520|
|   5809912| 5.24|grattol|1301|
|   5700037|  0.4| runail|1141|
|   5686925| 0.35|   null|1123|
|   5751422|10.95|    uno|1107|
|   5815662| 0.92|   null|1042|
|   5802432| 0.32|   null|1024|
|      5304| 0.32| runail| 925|
|   5792800|10.32|   null| 892|
|   5751383|10.32|    uno| 881|
|   5849033|10.32|    uno| 857|
|   5816170| 5.24|grattol| 717|
|   5687151|  1.9|   null| 693|
|   5809911| 5.24|grattol| 648|
|   5843836| 0.38|   null| 615|
|      5013| 1.27| runail| 606|
|   5793704| 2.38|  irisk| 559|
|   5561044|  3.0|   null| 544|
|   5528035| 9.52|   null| 524|
+----------+-----+-------+----+
only showing top 20 rows



### final query

In [ ]:
# 10월 코호트의 상품 구매리스트에서 최다 구매 상품 추출
result = spark.sql("""
WITH purch_list AS (
  SELECT event_time
      , product_id
      , brand
      , price
      , DATE_FORMAT(event_time, 'yyyy-MM-dd') AS time_format
      , DATE_FORMAT(MIN(event_time) OVER (PARTITION BY user_id), 'yyyy-MM-dd') AS first_order
  FROM total
  WHERE event_type = 'purchase'
)
SELECT product_id
    , brand
    , price
    , COUNT(*) AS sold
FROM purch_list
WHERE DATE_FORMAT(first_order, 'yyyy-MM') = '2019-10'
GROUP BY product_id, brand, price
ORDER BY sold DESC;
""")
result.show()

+----------+---------+-----+----+
|product_id|    brand|price|sold|
+----------+---------+-----+----+
|   5809910|  grattol| 5.24|1428|
|   5854897|    irisk| 0.32|1257|
|   5751422|      uno|10.95|1251|
|   5700037|   runail|  0.4|1158|
|   5802432|     null| 0.32|1105|
|   5792800|     null|10.32|1047|
|   5849033|      uno|10.32| 991|
|   5751383|      uno|10.32| 969|
|   5815662|     null| 0.92| 932|
|   5686925|     null| 0.35| 844|
|   5809912|  grattol| 5.24| 826|
|      5304|   runail| 0.32| 778|
|   5700046|   runail|  0.4| 640|
|   5528035|     null| 9.52| 632|
|   5687151|     null|  1.9| 615|
|      5013|   runail| 1.27| 574|
|   5688124|     null| 0.32| 543|
|   5793704|    irisk| 2.38| 499|
|   5833330|bpw.style| 0.95| 498|
|   5649219|     null| 7.14| 496|
+----------+---------+-----+----+
only showing top 20 rows



In [ ]:
# 1월 코호트의 상품 구매리스트에서 최다 구매 상품 추출
# count(id)로 함
result = spark.sql("""
WITH purch_list AS (
  SELECT event_time
      , product_id
      , brand
      , price
      , DATE_FORMAT(event_time, 'yyyy-MM-dd') AS time_format
      , DATE_FORMAT(MIN(event_time) OVER (PARTITION BY user_id), 'yyyy-MM-dd') as first_order
  FROM total
  WHERE event_type = 'purchase'
)
SELECT product_id
    , brand
    , price
    , COUNT(*) as sold
FROM purch_list
WHERE DATE_FORMAT(first_order, 'yyyy-MM') = '2020-01'
GROUP BY product_id, brand, price
ORDER BY sold DESC;
""")
result.show()

+----------+--------+-----+----+
|product_id|   brand|price|sold|
+----------+--------+-----+----+
|   5809910| grattol| 5.24|1409|
|   5854897|   irisk| 0.32| 792|
|   5686925|    null| 0.35| 723|
|   5809912| grattol| 5.24| 645|
|   5700037|  runail|  0.4| 585|
|   5751422|     uno|10.95| 522|
|      5304|  runail| 0.32| 516|
|   5802432|    null| 0.32| 498|
|   5815662|    null| 0.92| 489|
|   5751383|     uno|10.32| 435|
|   5849033|     uno|10.32| 391|
|   5816170| grattol| 5.24| 368|
|   5792800|    null|10.32| 360|
|   5809911| grattol| 5.24| 319|
|   5843836|    null| 0.38| 315|
|   5729864|    null| 0.41| 291|
|   5687151|    null|  1.9| 285|
|      5013|  runail| 1.27| 285|
|   5790689|nitrimax| 3.97| 284|
|   5528035|    null| 9.52| 271|
+----------+--------+-----+----+
only showing top 20 rows



In [ ]:
# 11월
result = spark.sql("""
WITH purch_list AS (
  SELECT event_time
      , product_id
      , brand
      , price
      , DATE_FORMAT(event_time, 'yyyy-MM-dd') AS time_format
      , DATE_FORMAT(MIN(event_time) OVER (PARTITION BY user_id), 'yyyy-MM-dd') as first_order
  FROM total
  WHERE event_type = 'purchase'
)
SELECT product_id
    , brand
    , price
    , COUNT(*) as sold
FROM purch_list
WHERE DATE_FORMAT(first_order, 'yyyy-MM') = '2019-11'
GROUP BY product_id, brand, price
ORDER BY sold DESC;
""")
result.show()

+----------+---------+-----+----+
|product_id|    brand|price|sold|
+----------+---------+-----+----+
|   5809910|  grattol| 5.24|1522|
|   5802432|     null| 0.32| 866|
|   5854897|    irisk| 0.32| 837|
|   5700037|   runail|  0.4| 663|
|   5815662|     null| 0.92| 651|
|   5751422|      uno|10.95| 630|
|   5809912|  grattol| 5.24| 619|
|      5304|   runail| 0.32| 617|
|   5751383|      uno|10.32| 563|
|   5792800|     null|10.32| 509|
|   5849033|      uno|10.32| 499|
|   5686925|     null| 0.35| 499|
|   5833330|bpw.style| 0.95| 420|
|   5809910|  grattol| 4.86| 415|
|   5816170|  grattol| 5.24| 394|
|   5833330|bpw.style| 0.81| 387|
|   5687151|     null|  1.9| 379|
|   5854897|    irisk|  0.3| 374|
|   5843836|     null| 0.38| 345|
|   5833326|bpw.style| 1.59| 337|
+----------+---------+-----+----+
only showing top 20 rows



In [ ]:
# 12월
result = spark.sql("""
WITH purch_list AS (
  SELECT event_time
      , product_id
      , brand
      , price
      , DATE_FORMAT(event_time, 'yyyy-MM-dd') AS time_format
      , DATE_FORMAT(MIN(event_time) OVER (PARTITION BY user_id), 'yyyy-MM-dd') as first_order
  FROM total
  WHERE event_type = 'purchase'
)
SELECT product_id
    , brand
    , price
    , COUNT(*) as sold
FROM purch_list
WHERE DATE_FORMAT(first_order, 'yyyy-MM') = '2019-12'
GROUP BY product_id, brand, price
ORDER BY sold DESC;
""")
result.show()

+----------+----------+-----+----+
|product_id|     brand|price|sold|
+----------+----------+-----+----+
|   5809910|   grattol| 5.24|1393|
|   5854897|     irisk| 0.32| 700|
|   5802432|      null| 0.32| 626|
|   5809912|   grattol| 5.24| 522|
|   5700037|    runail|  0.4| 509|
|      5304|    runail| 0.32| 489|
|   5815662|      null| 0.92| 445|
|   5751422|       uno|10.95| 421|
|   5751383|       uno|10.32| 364|
|   5833330| bpw.style| 0.95| 362|
|   5809911|   grattol| 5.24| 325|
|   5849033|       uno|10.32| 317|
|   5792800|      null|10.32| 309|
|   5843836|      null| 0.38| 304|
|   5528035|      null| 9.52| 286|
|   5729864|      null| 0.41| 271|
|      5013|    runail| 1.27| 264|
|   5759492|   italwax| 2.37| 259|
|   5816170|   grattol| 5.24| 250|
|   5776130|art-visage| 1.98| 229|
+----------+----------+-----+----+
only showing top 20 rows



In [ ]:
result = spark.sql("""
WITH purch_list AS (
  SELECT product_id
      , brand
      , price
      , event_time
      , DATE_FORMAT(event_time, 'yyyy-MM-dd') AS time_format
      , DATE_FORMAT(MIN(event_time) OVER (PARTITION BY user_id), 'yyyy-MM-dd') as first_order
  FROM total
  WHERE event_type = 'purchase'
)
SELECT product_id
    , brand
    , price
    , COUNT(*) as sold
FROM purch_list
WHERE DATE_FORMAT(first_order, 'yyyy-MM') = '2020-02'
GROUP BY product_id, brand, price
ORDER BY sold DESC;
""")
result.show()

+----------+---------+-----+----+
|product_id|    brand|price|sold|
+----------+---------+-----+----+
|   5809910|  grattol| 5.24| 849|
|   5854897|    irisk| 0.32| 578|
|   5802432|     null| 0.32| 437|
|   5700037|   runail|  0.4| 434|
|   5815662|     null| 0.92| 404|
|      5304|   runail| 0.32| 401|
|   5751422|      uno|10.95| 335|
|   5809912|  grattol| 5.24| 331|
|   5751383|      uno|10.32| 282|
|   5849033|      uno|10.32| 242|
|   5688124|     null| 0.32| 241|
|   5759492|  italwax| 2.37| 225|
|   5862943|freedecor| 0.79| 222|
|      5013|   runail| 1.27| 222|
|   5816170|  grattol| 5.24| 220|
|   5843836|     null| 0.38| 216|
|   5809911|  grattol| 5.24| 208|
|   5729864|     null| 0.41| 199|
|   5792800|     null|10.32| 194|
|   5815665|     null| 0.67| 192|
+----------+---------+-----+----+
only showing top 20 rows



In [ ]:
# 브랜드별 평균 가격과 전체 판매수
result = spark.sql("""
WITH purch_list AS (
  SELECT product_id
      , brand
      , price
  FROM total
  WHERE event_type = 'purchase'
)
SELECT brand
    , round(AVG(price),3) AS avg_price
    , COUNT(*) AS sold
FROM purch_list
GROUP BY brand
ORDER BY sold DESC;
""")
result.show()

+---------+---------+------+
|    brand|avg_price|  sold|
+---------+---------+------+
|     null|    4.666|549567|
|   runail|    3.083|111408|
|    irisk|    3.034| 73806|
|   masura|    2.809| 49751|
|  grattol|    5.375| 49542|
|bpw.style|    1.176| 37943|
| ingarden|    4.546| 27411|
|    estel|    6.265| 19438|
|      uno|   10.845| 17586|
|freedecor|      1.3| 17467|
|  italwax|    6.083| 17105|
|   kapous|    3.846| 16335|
|    domix|    3.852| 12958|
|     milv|    1.771| 12691|
| haruyama|    3.906| 11582|
|     pole|    3.576| 11578|
|  bluesky|    4.186| 11509|
| jessnail|    13.95|  9661|
|  concept|     5.63|  9530|
| severina|    2.864|  9173|
+---------+---------+------+
only showing top 20 rows



In [ ]:
# 2019 10월 코호트의 구매리스트에서 판매 상위 브랜드
result = spark.sql("""
WITH purch_list AS (
  SELECT event_time
      , product_id
      , brand
      , price
      , DATE_FORMAT(event_time, 'yyyy-MM-dd') AS time_format
      , DATE_FORMAT(MIN(event_time) OVER (PARTITION BY user_id), 'yyyy-MM-dd') as first_order
  FROM total
  WHERE event_type = 'purchase'
)
SELECT brand
    , COUNT(*) as sold
FROM purch_list
WHERE DATE_FORMAT(first_order, 'yyyy-MM') = '2019-10'
GROUP BY brand
ORDER BY sold DESC;
""")
result.show()

+---------+------+
|    brand|  sold|
+---------+------+
|     null|185717|
|   runail| 37726|
|    irisk| 25489|
|   masura| 17730|
|bpw.style| 15896|
|  grattol| 15334|
| ingarden| 10139|
|      uno|  6595|
|freedecor|  6145|
|    estel|  5450|
|     pole|  5094|
|   kapous|  5017|
|  italwax|  5015|
|    domix|  4778|
|     milv|  4599|
| haruyama|  4379|
|  bluesky|  3961|
|   zinger|  3385|
| jessnail|  3260|
|      cnd|  2932|
+---------+------+
only showing top 20 rows



In [ ]:
# 2020 1월 코호트의 구매리스트에서 판매 상위 브랜드
result = spark.sql("""
WITH purch_list AS (
  SELECT event_time
      , product_id
      , brand
      , price
      , DATE_FORMAT(event_time, 'yyyy-MM-dd') AS time_format
      , DATE_FORMAT(MIN(event_time) OVER (PARTITION BY user_id), 'yyyy-MM-dd') as first_order
  FROM total
  WHERE event_type = 'purchase'
)
SELECT brand
    , COUNT(*) as sold
FROM purch_list
WHERE DATE_FORMAT(first_order, 'yyyy-MM') = '2020-01'
GROUP BY brand
ORDER BY sold DESC;
""")
result.show()

+---------+-----+
|    brand| sold|
+---------+-----+
|     null|80378|
|   runail|18902|
|    irisk|11314|
|   masura| 9015|
|  grattol| 8134|
| ingarden| 3454|
|    estel| 3389|
|bpw.style| 3077|
|  italwax| 2990|
|      uno| 2435|
|   kapous| 2403|
|freedecor| 2100|
|    domix| 1928|
|  bluesky| 1833|
|     milv| 1640|
| severina| 1573|
|   zinger| 1516|
| haruyama| 1476|
| nagaraku| 1352|
|  concept| 1324|
+---------+-----+
only showing top 20 rows

